<a href="https://colab.research.google.com/github/pritamgouda11/Modelling-Indian_Names/blob/main/Gouda_Pritam_Trilochan_Savita_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2: Language Modelling (TA: Navreet Kaur)

The goal of this assignment is to model Indian first names using character-level language models.

You'll have to add your code wherever you see the comment `# ADD YOUR CODE HERE`. Please make a copy of this assignment, and you can use Google Colab notebooks to work on this. Later, you can download this notebook as a python file and submit it as per the following instructions.

**Important Note**: Whenever you add your code, please enclose your code block within two comments `# BEGIN CODE` and `# END CODE` to indicate the start and end of your code.
Example:

```
def model():

    data = get_dataloader() # this is defined by us

    # ADD YOUR CODE HERE

    # BEGIN CODE

    run_training() # this is written by you

    generate_names() # this is written by you

    # END CODE

```

## Submission Instructions

Note that your assignment will be based on the results obtained by functions implemented by you. Do not change the print statements having `EVALUATION` as they are used for auto-evaluation.

You'd need to submit the following files:

  * Save this `.ipynb` to your drive, complete the required code blocks and run the notebook.

  * After completing this assignment, download the notebook as`.py`. Name it as `SAPname_SRno_assignment2.py`, where `SAPname` refers to your name as per SAP record, and `SRno` refers to the last 5 digits of your IISc SR number. For example, IISc student with SAP name Twyla Linda (SR no - 04-03-00-10-22-20-1-15329) would name it as `Twyla_Linda_15329_assignment2.py`.

  *   The files associated with FNN and RNN models i.e. `model.pt`, `vocab.pt` and `loss.json` will be downloaded and saved in folders with names `fnn` and `rnn` respectively, and these will be contained within folder `SAPname_SRno`. Zip this folder along with the `.py` file, save it as `SAPname_SRno_assigment2` and upload on MS Teams. The directory can be summarised as follows:

```
SAPname_SRno_assigment2
│   SAPname_SRno_assignment2.py  
│
└───SAPname_SRno
│   │
│   └───fnn
│   │   │   model.pt
│   │   │   vocab.pt
│   │   │   loss.json
│   │
│   └───rnn
│       │   model.pt
│       │   vocab.pt
│       │   loss.json

```

Because submissions are auto-graded, please ensure that the naming of the files is consistent with the instructions.

## Broad Outline

In this assignment, we will implement different types of language models for modeling Indian names. There are clealry patterns in Indian names that models could learn, and we start modeling those using n-gram models, then move to neural n-gram and RNN models.


**Marks Distribution:**
- Unigram: 5
- Bigram: 15
- Trigram: 10
- Neural N-gram LM: 30
- RNN LM: 40



# Read and Preprocess Data

In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np
import math
import copy
import random
from collections import Counter, defaultdict

from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Please note that we may change the contents of the following four files when we rerun your code, so please make sure that your solution is not specifically engineered to just these names.

In [ ]:
# Download the training and validation datasets
!wget -O train_data.csv "https://docs.google.com/spreadsheets/d/1AUzwOQQbAehg_eoAMCcWfwSGhKwSAtnIzapt2wbv0Zs/gviz/tq?tqx=out:csv&sheet=train_data.csv"
!wget -O valid_data.csv "https://docs.google.com/spreadsheets/d/1UtQErvMS-vcQEwjZIjLFnDXlRZPxgO1CU3PF-JYQKvA/gviz/tq?tqx=out:csv&sheet=valid_data.csv"

# Download the text for evaluation
!wget -O eval_prefixes.txt "https://drive.google.com/uc?export=download&id=1tuRLJXLd2VcDaWENr8JTZMcjFlwyRo60"
!wget -O eval_sequences.txt "https://drive.google.com/uc?export=download&id=1kjPAR04UTKmdtV-FJ9SmDlotkt-IKM3b"

--2024-02-24 06:13:14--  https://docs.google.com/spreadsheets/d/1AUzwOQQbAehg_eoAMCcWfwSGhKwSAtnIzapt2wbv0Zs/gviz/tq?tqx=out:csv&sheet=train_data.csv
Resolving docs.google.com (docs.google.com)... 74.125.195.113, 74.125.195.100, 74.125.195.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘train_data.csv’

train_data.csv          [ <=>                ]  71.07K  --.-KB/s    in 0.005s  

2024-02-24 06:13:14 (15.1 MB/s) - ‘train_data.csv’ saved [72776]

--2024-02-24 06:13:14--  https://docs.google.com/spreadsheets/d/1UtQErvMS-vcQEwjZIjLFnDXlRZPxgO1CU3PF-JYQKvA/gviz/tq?tqx=out:csv&sheet=valid_data.csv
Resolving docs.google.com (docs.google.com)... 74.125.195.113, 74.125.195.100, 74.125.195.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving

In [ ]:
## Please do not change anything in this code block.

def read_dataframe(ds_type):
    """
    Args:
        ds_type [str] :  dataset type (train or valid)

    Returns:
        df [pandas dataframe]
    """

    df = pd.read_csv(f"/content/{ds_type}_data.csv", header=0, index_col=0)
    df = df[~df['Name'].isna()]
    df['Name'] = df['Name'].astype(str)
    return df

# Load the training and validation datasets
train_data = read_dataframe("train")
validation_data = read_dataframe("valid")

# Read files containing prefixes and character sequences for evaluation
with open('eval_prefixes.txt', 'r') as file:
    eval_prefixes = []
    for line in file:
        eval_prefixes.append(line.strip().split(" "))

with open('eval_sequences.txt', 'r') as file:
    eval_sequences = []
    for line in file:
        eval_sequences.append(line.strip().split(" "))

print(f"Length of training data: {len(train_data)}\nLength of validation data: {len(validation_data)}")

Length of training data: 4539
Length of validation data: 1297


In [ ]:
## Please do not change anything in this code block.

START = "<s>"   # Start-of-name token
END = "</s>"    # End-of-name token
UNK = "<unk>"   # token representing out of unknown (or out of vocabulary) tokens
vocab_from_ascii = True

def build_vocab(names):
    """
    Builds a vocabulary given a list of names

    Args:
        names [list[str]]: list of names

    Returns:
        vocab [torchtext.vocab]: vocabulary based on the names

    """

    if vocab_from_ascii:
        char_counts = {chr(i):i for i in range(128)}
    else:
        char_counts = Counter("".join(names))

    vocab = build_vocab_from_iterator(
                    char_counts,
                    specials=[UNK, START, END], #adding special tokens to the vocabulary
                    min_freq=1
                )
    vocab.set_default_index(vocab[UNK])
    return vocab


def tokenize_name(name):
    """
    Tokenise the name i.e. break a name into list of characters

    Args:
        name [str]: name to be tokenized

    Returns:
        list of characters
    """

    return list(str(name))


def process_data_for_input(data_iter, vocab):
    """
    Processes data for input: Breaks names into characters,
    converts out of vocabulary tokens to UNK and
    appends END token at the end of every name

    Args:
        data_iter: data iterator consisting of names
        vocab: vocabulary

    Returns:
        data_iter [list[list[str]]]: list of names, where each name is a
                                list of characters and is appended with
                                START and END tokens

    """

    vocab_set = set(vocab.get_itos())
    # convert Out Of Vocabulary (OOV) tokens to UNK tokens
    data_iter = [[char if char in vocab_set else UNK
                        for char in tokenize_name(name)] for name in data_iter]
    data_iter = [[START] + name + [END] for name in data_iter]

    return data_iter


def get_tokenised_text_and_vocab(ds_type, vocab=None):
    """
    Reads input data, tokenizes it, builds vocabulary (if unspecified)
    and outputs tokenised list of names (which in turn is a list of characters)

    Args:
        ds_type [str]: Type of the dataset (e.g., train, validation, test)
        vocab [torchtext.vocab]: vocabulary;
                                 If vocab is None, the function will
                                 build the vocabulary from input text.
                                 If vocab is provided, it will tokenize name
                                 according to the vocab, replacing any tokens
                                 not part of the vocab with UNK token.

    Returns:
        data_iter: data iterator for tokenized names
        vocab: vocabulary

    """

    # read the 'Name' column of the dataframe
    if ds_type=='train':
        data_iter = train_data['Name']
    elif ds_type=='valid':
        data_iter = validation_data['Name']
    else:
        data_iter = test_data['Name']

    # build vocab from input data, if vocab is unspecified
    if vocab is None:
        vocab = build_vocab(data_iter)

    # convert OOV chars to UNK, append START and END token to each name
    data_iter = process_data_for_input(data_iter, vocab)

    return data_iter, vocab

Let's look at some examples from the training set

In [ ]:
# Look at some random examples from the training set
examples = ", ".join(random.sample(list(train_data['Name']), 5))
f"Examples from the training set: {examples}"

'Examples from the training set: deepender, gollu, tushar, amrudin, satyaprakash'

# Module 1: N-gram Language Modelling

Load and preprocess the data for n-gram models

In [ ]:
"""choose your hyperparameter and see the difference in performance"""

# CHANGE THE None VALUES TO YOUR DESIRED VALUES

# ADD YOUR CODE HERE

# BEGIN CODE
names = train_data['Name']
name_lengths = names.str.len()
max_length = name_lengths.max()
mean_length = name_lengths.mean()
print("Mean length of names:", mean_length)
print("Maximum length of names:", max_length)
# END CODE

MAX_NAME_LENGTH = 11 # maximum length of names for generation

Mean length of names: 6.27605199383124
Maximum length of names: 19


In [ ]:
# Get data iterator and build vocabulary from input text

train_text, vocab = get_tokenised_text_and_vocab(ds_type='train')
validation_text, _ = get_tokenised_text_and_vocab(ds_type='valid', vocab=vocab)

# Check the size of vocabulary
vocab_size = len(vocab.get_stoi())
print(vocab_size)

131



Now it's time to implement an [n-gram language model](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

One edge case you will need to handle is that you don't have $n-1$ prior characters at the beginning of the text.  One way to do this is by appending the `START` token $n-1$ times at the start of the name, when implementing an $n$-gram model. You may choose whichever method you like to handle this case as long as you produce a valid probability distribution (one that sums to one).

**Generating names**

To generate from a language model, we can sample one char at a time conditioning on the chars we have generated so far.

In fact there are many strategies to get better-sounding samples, such as only sampling from the top-k chars or sharpening the distribution with a temperature.  You can read more about sampling from a language model in [this](https://arxiv.org/pdf/1904.09751.pdf) paper.


We will now implement N-gram models with $N=1$ (unigram), $N=2$ (bigram), and $N=3$ (trigram).

**Utility Functions**

Implement the utility functions `get_unigram_counts`, `get_bigram_counts` and `get_trigram_counts`. You can use these functions while implementing n-gram models.

In [ ]:
def get_unigram_counts(corpus):
    """
    Given a corpus, calculates the unigram counts for each character in the corpus

    Args:
        corpus [list[list[str]]]: list of tokenized characters. Text is appended with END token.

    Returns:
        unigram_counts [dict [key: char, value: count]]:
            dictionary of unigram counts for each character in the corpus
        Example:
        > unigram_counts["c1"] = 5
    """

     # ADD YOUR CODE HERE
     #BEGIN CODE
    unigram_counts ={}
    for name in corpus:
        for char in name:
            if char in unigram_counts:
                unigram_counts[char]= unigram_counts[char] +1
            else:
                unigram_counts[char]=1
      #END CODE
    return unigram_counts

In [ ]:
def get_bigram_counts(corpus):
    """
    Given a corpus, calculates the bigram counts for each bigram in the corpus.
    The corpus *only* contains END tokens at the end of names.
    You may want to handle the case where the beginning of the name
    does not have n-1 prior chars.

    Args:
        corpus [list[list[str]]]: list of tokenized text. Text is appended with END token.

    Returns:
        bigram_counts [dict[dict]]:
            nested dictionary of bigram counts for each bigram in the corpus
        Example:
        > bigram_counts["c1"]["c2"] = 5
        here bigram_counts["c1"]["c2"] represents P("c2"|"c1")
        P["c1"]["c2"] means P[char_i = "c2"|char_{i-1} = "c1"]
    """

    bigram_counts = {}

    for name in corpus:

        padded_name = ["<s>"] * (len(name) - 1) + name
        for i in range(1, len(padded_name)):
            bigram = (padded_name[i-1], padded_name[i])
            if bigram[0] not in bigram_counts:
                bigram_counts[bigram[0]] = {}
            bigram_counts[bigram[0]][bigram[1]] = bigram_counts[bigram[0]].get(bigram[1], 0) + 1

    return bigram_counts


In [ ]:
def get_trigram_counts(corpus):
    """
    Given a corpus, calculates the trigram counts for each trigram in the corpus.
    The corpus *only* contains END tokens at the end of names.
    You may want to handle the case where the beginning of the text
    does not have n-1 prior chars.

    Args:
        corpus [list[list[str]]]: list of tokenized text. Text is appended with END token.

    Returns:
        trigram_counts [dict[dict[dict]]]:
            nested dictionary for each trigram in the corpus
        Example:
        > trigram_counts["c1"]["c2"]["c3"] = 5
        P["c1"]["c2"]["c3] means P[char_i = "c3"|char_{i-2} = "c1", char_{i-1} = "c2"]

    """

    trigram_counts = {}

    for name in corpus:
        padded_name = ["<s>"] * (len(name) - 2) + name
        for i in range(2, len(padded_name)):
            trigram = (padded_name[i-2], padded_name[i-1], padded_name[i])
            if trigram[0] not in trigram_counts:
                trigram_counts[trigram[0]] = {}

            if trigram[1] not in trigram_counts[trigram[0]]:
                trigram_counts[trigram[0]][trigram[1]] = {}

            trigram_counts[trigram[0]][trigram[1]][trigram[2]] = trigram_counts[trigram[0]][trigram[1]].get(trigram[2], 0) + 1

    return trigram_counts


In [ ]:
from collections import defaultdict, Counter
import random

class NGramLanguageModel(object):
    def __init__(self, train_text):
        """
        Initialize and train the model with train_text.

        Args:
            train_text [list of list]: list of tokenized names
        """
        self.train_text = train_text
        self.vocab = build_vocab(train_text)
        self.n = None  # Set n-gram order during training
        self.ngram_counts = defaultdict(Counter)  # Initialize n-gram counts

        # Train the model
        self.train(train_text)

    def train(self, train_text):
        """
        Train the language model with the given text.

        Args:
            train_text [list of list]: list of tokenized names
        """
        # Determine the n-gram order based on the length of the tokenized names
        max_name_length = max(len(name) for name in train_text)
        if max_name_length <= 1:
            self.n = 1  # Unigram model
        elif max_name_length == 2:
            self.n = 2  # Bigram model
        elif max_name_length == 3:
            self.n = 3  # Trigram model
        else:
            self.n = 3  # For simplicity, use trigram model for names longer than 3 characters

        # Count n-grams in the training text
        for name in train_text:
            self.count_ngrams(name)

    def count_ngrams(self, name):
        """
        Count n-grams in a given name.

        Args:
            name [list of str]: tokenized name
        """
        # Pad the name with start-of-sequence and end-of-sequence tokens
        padded_name = [START] * (self.n - 1) + name + [END]

        # Count n-grams
        for i in range(len(padded_name) - self.n + 1):
            ngram = tuple(padded_name[i:i + self.n])
            self.ngram_counts[ngram[:-1]][ngram[-1]] += 1

    def get_next_char_probabilities(self, *args):
        """
        Returns a probability distribution over all characters in the vocabulary
        given the previous character(s), based on the value of n.

        Args:
            *args: previous character(s)

        Returns:
            probabilities [dict]: probability distribution over characters
        """
        if self.n == 1:
            # Unigram model
            if len(args) != 0:
                raise ValueError("Unigram model does not require any previous character.")
            total_count = sum(self.ngram_counts[()].values())
            probabilities = {char: count / total_count for char, count in self.ngram_counts[()].items()}
        elif self.n == 2:
            # Bigram model
            if len(args) != 1:
                raise ValueError("Bigram model requires exactly one previous character.")
            char = args[0]
            if char not in self.ngram_counts:
                return {}
            total_count = sum(self.ngram_counts[char].values())
            probabilities = {next_char: count / total_count for next_char, count in self.ngram_counts[char].items()}
        elif self.n == 3:
            # Trigram model
            if len(args) != 2:
                raise ValueError("Trigram model requires exactly two previous characters.")
            prev_chars = args
            if prev_chars not in self.ngram_counts:
                return {}
            total_count = sum(self.ngram_counts[prev_chars].values())
            probabilities = {char: count / total_count for char, count in self.ngram_counts[prev_chars].items()}
        else:
            raise ValueError("Unsupported value of n.")
        return probabilities

    def get_name_log_probability(self, name):
        """
        Calculates the log probability of name according to the language model

        Args:
            name [list]: list of tokens

        Returns:
            log_prob [float]: Log probability of the given name
        """
        log_prob = 0.0
        padded_name = [START] * (self.n - 1) + name + [END]
        for i in range(len(padded_name) - self.n + 1):
            ngram = tuple(padded_name[i:i + self.n])
            prev_chars = ngram[:-1]
            next_char = ngram[-1]
            if prev_chars not in self.ngram_counts or next_char not in self.ngram_counts[prev_chars]:
                return float('-inf')  # Return negative infinity for unknown n-grams
            log_prob += math.log(self.get_next_char_probabilities(*prev_chars)[next_char])
        return log_prob

    def get_perplexity(self, text):
        """
        Returns the perplexity of the model on a text as a float.

        Args:
            text [list]: a list of string tokens

        Returns:
            perplexity [float]: perplexity of the given text
        """
        # Calculate the log probability of the text
        log_prob_sum = sum(self.get_name_log_probability(name) for name in text)

        # Calculate perplexity using log probabilities
        perplexity = math.exp(-log_prob_sum / len(text))
        return perplexity

    def generate_names(self, k, n=MAX_NAME_LENGTH, prefix=None):
        """
        Given a prefix, generate k names according to the model.
        The default prefix is None.
        You may stop the generation when n tokens have been generated,
        or when you encounter the END token.

        Args:
            k [int]: Number of names to generate
            n [int]: Maximum length (number of tokens) in the generated name
            prefix [list of tokens]: Prefix after which the names have to be generated

        Returns:
            names [list[str]]: list of generated names
        """
        names = []
        for _ in range(k):
            generated_name = []
            if prefix is None:
                prefix = [START] * (self.n - 1)
            else:
                prefix = [START] * (self.n - len(prefix)) + prefix
            for _ in range(n):
                next_char_probabilities = self.get_next_char_probabilities(*prefix)
                if not next_char_probabilities:
                    break
                next_char = random.choices(list(next_char_probabilities.keys()),
                                           list(next_char_probabilities.values()))[0]
                if next_char == END:
                    break
                generated_name.append(next_char)
                prefix = (prefix + [next_char])[1:]
            names.append(''.join(generated_name))
        return names

    def get_most_likely_chars(self, sequence, k):
        """
        Given a sequence of characters, outputs k most likely characters after the sequence.

        Args:
            sequence [list[str]]: list of characters
            k [int]: number of most likely characters to return

        Returns:
            chars [list[str]]: *Ordered* list of most likely characters
                        (with charcater at index 0 being the most likely and
                        character at index k-1 being the least likely)

        """
        prefix = sequence[-(self.n - 1):] if len(sequence) >= self.n - 1 else [START] * (self.n - len(sequence)) + sequence
        next_char_probabilities = self.get_next_char_probabilities(*prefix)
        most_likely_chars = sorted(next_char_probabilities, key=lambda x: next_char_probabilities[x], reverse=True)[:k]
        return most_likely_chars


In [ ]:
## Please do not change anything in this code block.

def check_validity(model, ngram, is_neural):
    """
    Checks if get_next_char_probabilities returns a valid probability distribution
    """

    if ngram==1 or is_neural:
        P = model.get_next_char_probabilities()
        is_valid = validate_probability_distribution(P.values())
        if not is_valid:
            return is_valid

    elif ngram==2:
        P = model.get_next_char_probabilities()
        for char1 in P.keys():
            is_valid = validate_probability_distribution(list(P[char1].values()))
            if not is_valid:
                return is_valid

    elif ngram==3:
        P = model.get_next_char_probabilities()
        for char1 in P.keys():
            for char2 in P[char1].keys():
                is_valid = validate_probability_distribution(list(P[char1][char2].values()))
                if not is_valid:
                    return is_valid
    else:
        print("Enter a valid number for ngram")

    return True


def validate_probability_distribution(probs):
    """
    Checks if probs is a valid probability distribution
    """
    if not min(probs) >= 0:
        print("Negative value in probabilities")
        return False
    elif not max(probs) <= 1 + 1e-8:
        print("Value larger than 1 in probabilities")
        return False
    elif not abs(sum(probs)-1) < 1e-4:
        print("probabilities do not sum to 1")
        return False
    return True


def eval_ngram_model(model, ngram, ds, ds_name, eval_prefixes, eval_sequences, num_names=5, is_neural=False):
    """
    Runs the following evaluations on n-gram models:
    (1) checks if probability distribution returned by model.get_next_char_probabilities() sums to one
    (2) checks the perplexity of the model
    (3) generates names using model.generate_names()
    (4) generates names given a prefix using model.generate_names()
    (4) output most likely characters after a given sequence of chars using model.get_most_likely_chars()
    """

    # (1) checks if probability distributions sum to one
    is_valid = check_validity(model=model, ngram=ngram, is_neural=is_neural)
    print(f'EVALUATION probability distribution is valid: {is_valid}')

    # (2) evaluate the perplexity of the model on the dataset
    print(f'EVALUATION of {ngram}-gram on {ds_name} perplexity:',
        model.get_perplexity(ds))

    # (3) generate a few names
    generated_names = ", ".join(model.generate_names(k=num_names))
    print(f'EVALUATION {ngram}-gram generated names are {generated_names}')

    # (4) generate a few names given a prefix
    for prefix in eval_prefixes:
        generated_names_with_prefix = ", ".join(model.generate_names(k=num_names, prefix=prefix))
        prefix = ''.join(prefix)
        print(f'EVALUATION {ngram}-gram generated names with prefix {prefix} are {generated_names_with_prefix}')

    # (5) get most likely characters after a sequence
    for sequence in eval_sequences:
        most_likely_chars = ", ".join(model.get_most_likely_chars(sequence=sequence, k=num_names))
        sequence = "".join(sequence)
        print(f"EVALUATION {ngram}-gram top most likely chars after {sequence} are {most_likely_chars}")

## 1.1 Unigram

In [ ]:
class UnigramModel(NGramLanguageModel):
    def __init__(self, train_text):
        """
        Initialize and train the model with train_text.

        Args:
            train_text [list of list]: list of tokenized names
        """
        super().__init__(train_text)
        self.ngram_counts[()] = get_unigram_counts(train_text)

    def get_next_char_probabilities(self):
        """
        Return a dictionary of probabilities for each char in the vocabulary

        Returns:
            key: char, value: probability
        """
        total_count = sum(self.ngram_counts[()].values())
        next_char_probabilities = {char: count / total_count for char, count in self.ngram_counts[()].items()}
        return next_char_probabilities

    def get_name_log_probability(self, name):
        """
        Calculates the log probability of name according to the n-gram model

        Args:
            name [list]: list of tokens

        Returns:
            Log probability of the name [float]
        """
        log_prob = sum(math.log(self.get_next_char_probabilities().get(char, 1e-10)) for char in name)
        return log_prob

    def get_perplexity(self, text):
        """
        Returns the perplexity of the model on a text as a float.

        Args:
            text [list]: a list of string tokens

        Returns:
            perplexity of the given text [float]
        """
        log_prob_sum = sum(self.get_name_log_probability(name) for name in text)
        perplexity = math.exp(-log_prob_sum / sum(len(name) for name in text))
        return perplexity

    def generate_names(self, k, n=MAX_NAME_LENGTH, prefix=None):
        """
        Given a prefix, generate k names according to the model.
        The default prefix is None.

        Args:
            k [int]: Number of names to generate
            n [int]: Maximum length (number of tokens) in the generated name
            prefix [list of tokens]: Prefix after which the names have to be generated

        Returns:
            list of generated names [list]
        """
        names = []
        for _ in range(k):
            generated_name = []
            char_probabilities = self.get_next_char_probabilities()
            while len(generated_name) < n:
                char = random.choices(list(char_probabilities.keys()), weights=list(char_probabilities.values()))[0]
                if char == END:
                    break
                generated_name.append(char)
            names.append(''.join(generated_name))  # Join characters into a single string
        return names

    def get_most_likely_chars(self, sequence, k):
        """
        Given a sequence of characters, outputs k most likely characters after the sequence.

        Args:
            sequence [list[str]]: list of characters
            k [int]: number of most likely characters to return

        Returns:
            chars [list[str]]: *Ordered* list of most likely characters
                        (with charcater at index 0 being the most likely and
                        character at index k-1 being the least likely)

        """
        char_probabilities = self.get_next_char_probabilities()
        sorted_chars = sorted(char_probabilities.keys(), key=lambda x: char_probabilities[x], reverse=True)
        return sorted_chars[:k]


In [ ]:
unigram_model = UnigramModel(train_text)
print(f"probs: {unigram_model.generate_names(5)}")

probs: ['lraa<s>dbpnen', 'a<s>naknpa<s>g<s>', 'ial<s>ahhesaa', 'ual', 'd<s>ilroo<s>aha']


### Eval

**Note**: For models without smoothing, you may observe perplexity as `inf` if the validation or test set contains characters not seen in the train set
However, this should not happen for models where you implement smoothing.

In [ ]:
## Please do not change anything in this code block.

unigram_model = UnigramModel(train_text)

# Check the perplexity of the unigram model on the train set
print('unigram train perplexity:',
      unigram_model.get_perplexity(train_text))

unigram train perplexity: 16.623900007096303


In [ ]:
## Please do not change anything in this code block.

eval_ngram_model(model=unigram_model, ngram=1, ds=validation_text, ds_name='validation', eval_prefixes=eval_prefixes, eval_sequences=eval_sequences, num_names=5)

EVALUATION probability distribution is valid: True
EVALUATION of 1-gram on validation perplexity: 53.21745105778266
EVALUATION 1-gram generated names are u, sdas<s>s, tpkajnpeidl, , edn<s>s
EVALUATION 1-gram generated names with prefix <s><s>sh are , <s>i, s, nso, 
EVALUATION 1-gram top most likely chars after <s><s>aa are a, <s>, </s>, i, n


### Smoothing

Implement a smoothed version of the unigram model. You may extend the `UnigramModel` class and re-use some of the functions.  For unigram model, you should implement Add-1 smoothing.

You may refer to the lecture slides or [3.5 Smoothing](https://web.stanford.edu/~jurafsky/slp3/3.pdf) for details on different smoothing technqiues.


In [ ]:
class SmoothedUnigramModel(UnigramModel):
    def __init__(self, train_text):
        super().__init__(train_text)
        # Additional step to ensure smoothing is applied to all vocab items
        self.adjust_counts_for_smoothing()

    def adjust_counts_for_smoothing(self):
        # Adjust the counts for each character in the vocab, including unseen characters
        for char in self.vocab.get_itos():
            self.ngram_counts[()][char] = self.ngram_counts[()].get(char, 0) + 1

        # Adjust the total count to account for the added counts
        self.total_count = sum(self.ngram_counts[()].values())

    def get_next_char_probabilities(self):
        # Calculate the probability of each character
        total_vocab_size = len(self.vocab.get_itos())  # Ensure this includes all possible characters
        next_char_probabilities = {char: count / self.total_count for char, count in self.ngram_counts[()].items()}
        return next_char_probabilities


In [ ]:
## Please do not change anything in this code block.

smoothed_unigram_model = SmoothedUnigramModel(train_text)

# Check the perplexity of the smoothed unigram model on the train set
print('smoothed unigram train perplexity:',
      smoothed_unigram_model.get_perplexity(train_text))

smoothed unigram train perplexity: 16.669321820978315


In [ ]:
## Please do not change anything in this code block.

eval_ngram_model(model=smoothed_unigram_model, ngram=1, ds=validation_text, ds_name='validation', eval_prefixes=eval_prefixes, eval_sequences=eval_sequences,  num_names=5)

EVALUATION probability distribution is valid: True
EVALUATION of 1-gram on validation perplexity: 25.488899811273047
EVALUATION 1-gram generated names are r<s>aaa<s>, k, kipt, , n<s>pauiua<s>s
EVALUATION 1-gram generated names with prefix <s><s>sh are j, aaslasija, dedyrmltsr<s>, , 
EVALUATION 1-gram top most likely chars after <s><s>aa are a, <s>, </s>, i, n


In [ ]:
# Release models we don't need any more.
del unigram_model
del smoothed_unigram_model

## 1.2 Bigram

In [ ]:
"""
Implementation of a Bigram Model.
"""

class BigramModel(NGramLanguageModel):
    def __init__(self, train_text):
        """
        Initialise and train the model with train_text.

        Args:
            train_text [list of list]: list of tokenised names
        """

        # ADD YOUR CODE HERE
        self.train_text =  train_text
        self.bi_char_counts = get_bigram_counts(train_text)
        vocab = dict()

        for key, value in self.bi_char_counts.items():
            vocab[key] = set(value.keys())
        self.vocab = vocab
        outer_key_sums = {key: sum(value.values()) for key, value in self.bi_char_counts.items()}
        prob = {}
        for outer_key, inner_dict in self.bi_char_counts.items():
            prob[outer_key] = {inner_key: value / outer_key_sums[outer_key] for inner_key, value in inner_dict.items()}
        self.char_probs = prob


    def get_next_char_probabilities(self):
        """
        Returns a probability distribution over all chars in the vocabulary.
        Probability distribution should sum to one.

        Returns:
            P: dictionary or nested dictionary; Output format depends on n-gram
            Examples:
                for N=2 (bigram); dict[key:bigram_char1, value:dict[key:bigram_char2,value:probability of bigram]]
                    > P["a"]["b"] = 0.0001 (which stands of P("b"|"a"))
        """

        # ADD YOUR CODE HERE
        outer_key_sums = {key: sum(value.values()) for key, value in self.bi_char_counts.items()}
        prob = {}
        for outer_key, inner_dict in self.bi_char_counts.items():
            prob[outer_key] = {inner_key: value / outer_key_sums[outer_key] for inner_key, value in inner_dict.items()}

        return prob


    def get_name_log_probability(self, name):
        """
        Calculates the log probability of name according to the n-gram model.
        Be careful with cases for which probability of the name is zero.

        Args:
            name [list]: list of tokens

        Returns:
            Log probability of the name [float]
        """

        # ADD YOUR CODE HERE
        log_prob = 0.0

        for i in range(len(name)-1):
            char1= name[i]
            char2 = name[i+1]
            if char1 in self.char_probs:
                if char2 in self.char_probs[char1]:
                    log_prob += np.log(self.char_probs[char1].get(char2,0))

        return log_prob



    def get_perplexity(self, text):
        """
        Returns the perplexity of the model on a text as a float.

        Args:
            text [list]: a list of string tokens

        Returns:
            perplexity of the given text [float]
        """

        # ADD YOUR CODE HERE
        log_prob =0.0
        n=0
        for name in text:
            log_prob += self.get_name_log_probability(name)
            n += len(name)

        perplexity = math.exp(-log_prob/n)
        return perplexity

    def sample_from_unigram(self,name,temperature):
        unigram_model = UnigramModel(train_text)
        return unigram_model.sample_next_char(name,temperature)

    def sample_next_char (self,name, temperature):
        prev_char = name[-1]
        next_char_probs = self.char_probs.get(prev_char, {})
        if temperature > 0.0:
            probs = [p ** (1 / temperature) for p in next_char_probs.values()]
            sum_probs = sum(probs)
            probs = [p / sum_probs for p in probs]
        else:
            probs = next_char_probs.values()

        # Sample a next character
        next_char = random.choices(list(next_char_probs.keys()), weights=probs)[0]

        # Ensure the sampled character is valid
        if next_char not in self.vocab[prev_char]:
            next_char = random.choice(list(self.vocab[prev_char]))

        return next_char



    def generate_names(self, k, n=MAX_NAME_LENGTH, prefix=None):
        """
        Given a prefix, generate k names according to the model.
        The default prefix is None.

        Args:
            k [int]: Number of names to generate
            n [int]: Maximum length (number of tokens) in the generated name
            prefix [list of tokens]: Prefix after which the names have to be generated

        Returns:
            list of generated names [list]
        """

        # ADD YOUR CODE HERE
        names= []
        for _ in range(k):
            if prefix:
                if prefix[-1] == '</s>':
                    prefix = prefix[:-1]
                name=prefix
            else:
                name=['<s>']
            while len(name) <n and name[-1]!='</s>':
                next_char = self.sample_next_char(name,0.5)
                if next_char == "<unk>":
                    next_char = self.sample_from_unigram(name,0.5)
                name.append(next_char)
            names.append(name)
        generated_name = [''.join(sublist) for sublist in names]
        return generated_name


    def get_most_likely_chars(self, sequence, k):
        """
        Given a sequence of characters, outputs k most likely characters after the sequence.

        Args:
            sequence [list[str]]: list of characters
            k [int]: number of most likely characters to return

        Returns:
            chars [list[str]]: *Ordered* list of most likely characters
                        (with charcater at index 0 being the most likely and
                        character at index k-1 being the least likely)

        """
        # ADD YOUR CODE HERE
        last_char = sequence[-1]
        next_char_probs = self.char_probs.get(last_char, {})
        valid_chars = self.vocab[last_char]
        next_char_probs = {char: prob for char, prob in next_char_probs.items() if char in valid_chars}
        sorted_chars = sorted(next_char_probs, key=next_char_probs.get, reverse=True)
        most_likely_chars = sorted_chars[:k]

        return most_likely_chars

### Eval

In [ ]:
## Please do not change anything in this code block.

bigram_model = BigramModel(train_text)

# check the perplexity of the bigram model on training data
print('bigram train perplexity:',
      bigram_model.get_perplexity(train_text))

bigram train perplexity: 9.886271551428392


In [ ]:
## Please do not change anything in this code block.

eval_ngram_model(model=bigram_model, ngram=2, ds=validation_text, ds_name='validation', eval_prefixes=eval_prefixes, eval_sequences=eval_sequences, num_names=5)

EVALUATION probability distribution is valid: True
EVALUATION of 2-gram on validation perplexity: 6.9449719290471075
EVALUATION 2-gram generated names are <s><s><s><s><s><s><s><s>bi</s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>
EVALUATION 2-gram generated names with prefix <s><s>shal</s> are <s><s>shal</s>, <s><s>shallalan, <s><s>shallalan, <s><s>shallalan, <s><s>shallalan
EVALUATION 2-gram top most likely chars after <s><s>aa are </s>, n, r, m, l


### Smoothing

Implement a smoothed version of the bigram model. You may extend the `BigramModel` class and re-use some of the functions.

You will implement the following smoothing techniques:
-  Laplace or add-k smoothing
- Interpolation


**Laplace or Add-k smoothing**
- what is the effect of changing `k`?

In [ ]:
"""choose your hyperparameter and see the difference in performance"""

# ADD YOUR CODE HERE

# CHANGE THE None VALUES TO YOUR DESIRED VALUES
# Please feel free to play with these hyperparameters to see the effects on the
# quality of generated names and perplexity

BIGRAM_LAPLACE_K = 0.9 # value of k for add-k or Laplac smoothing in bigram models

In [ ]:
"""
Implementation of a bigram model with laplace or add-k smoothing.

"""

class LaplaceSmoothedBigramModel(BigramModel):
    # This class extends BigramModel.

    def __init__(self, train_text, k):
        super().__init__(train_text)
        self.k = k # specify k for smoothing

    # You should override ONLY those functions
    # which calculate probability of a bigram.
    # You can override get_next_char_probabilities
    # or any other helper functions you use in BigramModel
    # to calculate bigram probabilities.

    # ADD YOUR CODE HERE

    def get_name_log_probability(self, name):
        log_prob = 0.0
        char_probs = self.get_next_char_probabilities()
        for i in range(len(name) - 1):
            char1 = name[i]
            char2 = name[i + 1]
            if char1 not in char_probs:
                continue
            if char2 not in char_probs[char1]:
                total = sum(char_probs[char1].values())
                log_prob += np.log(self.k/total + self.k*len(self.vocab[char1]))
            else:
                log_prob += np.log(char_probs[char1][char2])
        return log_prob

In [ ]:
## Please do not change anything in this code block.

smoothed_bigram_model = LaplaceSmoothedBigramModel(train_text, k=BIGRAM_LAPLACE_K)

# check the perplexity of the bigram model on training data
print('smoothed bigram train perplexity:',
      smoothed_bigram_model.get_perplexity(train_text))

smoothed bigram train perplexity: 9.886271551428392


In [ ]:
## Please do not change anything in this code block.

eval_ngram_model(model=smoothed_bigram_model, ngram=2, ds=validation_text, ds_name='validation', eval_prefixes=eval_prefixes, eval_sequences=eval_sequences, num_names=5)

EVALUATION probability distribution is valid: True
EVALUATION of 2-gram on validation perplexity: 5.78673835687362
EVALUATION 2-gram generated names are <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>
EVALUATION 2-gram generated names with prefix <s><s>shal</s> are <s><s>shali</s>, <s><s>shalishat, <s><s>shalishat, <s><s>shalishat, <s><s>shalishat
EVALUATION 2-gram top most likely chars after <s><s>aa are </s>, n, r, m, l


**Interpolation**
- what are good values for `lambdas` in interpolation?

In [ ]:
"""choose your hyperparameter and see the difference in performance"""

# ADD YOUR CODE HERE

# CHANGE THE None VALUES TO YOUR DESIRED VALUES
# Please feel free to play with these hyperparameters to see the effects on the
# quality of generated names and perplexity

BIGRAM_LAMBDAS = (0.9, 0.1) # lambdas for interpolation smoothing in bigram models

In [ ]:
"""
Implementation of a bigram model with interpolation smoothing
"""

class InterpolationSmoothedBigramModel(BigramModel):

    def __init__(self, train_text, lambdas):
        super().__init__(train_text)
        self.lambda_1, self.lambda_2 = lambdas

    # You should override ONLY those functions
    # which calculate probability of a bigram.
    # You can override get_next_char_probabilities
    # or any other helper functions you use in BigramModel
    # to calculate bigram probabilities.

    # ADD YOUR CODE HERE
    def get_name_log_probability(self, name):
        log_prob = 0.0
        char_probs = self.get_next_char_probabilities()
        for i in range(len(name) - 1):
            char1 = name[i]
            char2 = name[i + 1]
            if char1 not in char_probs or char2 not in char_probs[char1]:
                prob_char1 = 1 / len(self.vocab)  # uniform
                prob_char2_given_char1 = 1 / len(self.vocab)  # uniform  bigrams
                log_prob += np.log(self.lambda_1 * prob_char1 + self.lambda_2 * prob_char2_given_char1)
            else:
                log_prob += np.log(char_probs[char1][char2])
        return log_prob

In [ ]:
## Please do not change anything in this code block.

smoothed_bigram_model = InterpolationSmoothedBigramModel(train_text, lambdas=BIGRAM_LAMBDAS)

# check the perplexity of the bigram model on training data
print('smoothed bigram train perplexity:',
      smoothed_bigram_model.get_perplexity(train_text))

smoothed bigram train perplexity: 9.886271551428392


In [ ]:
## Please do not change anything in this code block.

eval_ngram_model(model=smoothed_bigram_model, ngram=2, ds=validation_text, ds_name='validation', eval_prefixes=eval_prefixes, eval_sequences=eval_sequences, num_names=5)

EVALUATION probability distribution is valid: True
EVALUATION of 2-gram on validation perplexity: 10.356411073095366
EVALUATION 2-gram generated names are <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>
EVALUATION 2-gram generated names with prefix <s><s>shal</s> are <s><s>shala</s>, <s><s>shalan</s>, <s><s>shalandi</s>, <s><s>shalandi</s>, <s><s>shalandi</s>
EVALUATION 2-gram top most likely chars after <s><s>aa are </s>, n, r, m, l


In [ ]:
print(1)
model = InterpolationSmoothedBigramModel(train_text, lambdas=(0.9, 0.1))
probabilities_1 = model.get_next_char_probabilities()
eval_ngram_model(model=smoothed_bigram_model, ngram=2, ds=validation_text, ds_name='validation', eval_prefixes=eval_prefixes, eval_sequences=eval_sequences, num_names=5)

print(2)
model = InterpolationSmoothedBigramModel(train_text, lambdas=(0.5, 0.5))
probabilities_2 = model.get_next_char_probabilities()
eval_ngram_model(model=smoothed_bigram_model, ngram=2, ds=validation_text, ds_name='validation', eval_prefixes=eval_prefixes, eval_sequences=eval_sequences, num_names=5)

print(3)
model = InterpolationSmoothedBigramModel(train_text, lambdas=(0.1, 0.9))
probabilities_3 = model.get_next_char_probabilities()
eval_ngram_model(model=smoothed_bigram_model, ngram=2, ds=validation_text, ds_name='validation', eval_prefixes=eval_prefixes, eval_sequences=eval_sequences, num_names=5)
char = 'a'  # Example character
print(probabilities_1.get(char))
print(probabilities_2.get(char))
print(probabilities_3.get(char))


1
EVALUATION probability distribution is valid: True
EVALUATION of 2-gram on validation perplexity: 10.356411073095366
EVALUATION 2-gram generated names are <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>
EVALUATION 2-gram generated names with prefix <s><s>shal</s> are <s><s>shal</s>, <s><s>shal</s>, <s><s>shalan</s>, <s><s>shalana</s>, <s><s>shalanar</s>
EVALUATION 2-gram top most likely chars after <s><s>aa are </s>, n, r, m, l
2
EVALUATION probability distribution is valid: True
EVALUATION of 2-gram on validation perplexity: 10.356411073095366
EVALUATION 2-gram generated names are <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>
EVALUATION 2-gram generated names with prefix <s><s>shal</s> are <s><s>shala</s>, <s><s>shala</s>, <s><s>shal

In [ ]:
# Release models we don't need any more.
del bigram_model
del smoothed_bigram_model

## 1.3 Trigram (smoothed)

In [ ]:
"""choose your hyperparameter and see the difference in performance"""

# ADD YOUR CODE HERE

# CHANGE THE None VALUES TO YOUR DESIRED VALUES
# Please feel free to play with these hyperparameters to see the effects on the
# quality of generated names and perplexity

TRIGRAM_LAMBDAS = (0.1, 0.3, 0.6) # lambdas for interpolation smoothing in trigram models
lambda_1=0.1
lambda_2=0.3
lambda_3=0.6

In [ ]:
"""
Implementaion of a Trigram Model with interpolation smoothing.
"""
class TrigramModel(NGramLanguageModel):
    def __init__(self, train_text):
        """
        Initialise and train the model with train_text.

        Args:
            train_text [list of list]: list of tokenised names
        """

        # ADD YOUR CODE HERE
        self.train_text = train_text
        self.char_counts = get_trigram_counts(train_text)
        vocab =get_trigram_counts(train_text)
        for key, value in vocab.items():
            for inner_key, inner_value in value.items():
                if isinstance(inner_value, dict):
                    vocab[key][inner_key] = set(inner_value.keys())
        self.vocab=vocab

        total_counts =get_trigram_counts(train_text)
        for outer_key,outer_val  in self.char_counts.items():
            #print(outer_val)
            for inner_key,inner_val in self.char_counts[outer_key].items():
                #print(inner_val)
                if isinstance(inner_val, dict):
                    total_counts[outer_key][inner_key] = sum(inner_val.values())
        self.total= total_counts

        char_probs = get_trigram_counts(train_text)
        for outer_key,outer_val in char_probs.items():
            #print(outer_val)
            for inner_key,inner_val in char_probs[outer_key].items():
                #print(inner_val)
                if isinstance(inner_val, dict):
                    for key,value in char_probs[outer_key][inner_key].items():
                        #print(value)
                        char_probs[outer_key][inner_key][key] = value/self.total[outer_key][inner_key]
        self.char_probs = char_probs




    def get_next_char_probabilities(self):
        """
        Returns a probability distribution over all chars in the vocabulary.
        Probability distribution should sum to one.

        Returns:
            P: dictionary or nested dictionary; Output format depends on n-gram
            Examples:
                for N=1 (unigram); dict[key:unigram,value:probability of unigram]
                    > P["a"] = 0.0001
                for N=2 (bigram); dict[key:bigram_char1, value:dict[key:bigram_char2,value:probability of bigram]]
                    > P["a"]["b"] = 0.0001 (corresponding to P(b|a))
                for N=3 (trigram); dict[dict[dict]]
                    > P["a"]["b"]["c"] = 0.0001 (corresponding to P(c|ab))
        """

        # ADD YOUR CODE HERE

        return self.char_probs


    def get_name_log_probability(self, name):
        """
        Calculates the log probability of name according to the n-gram model.
        Be careful with cases for which probability of the name is zero.

        Args:
            name [list]: list of tokens

        Returns:
            Log probability of the name [float]
        """

        # ADD YOUR CODE HERE
        log_prob = 0.0
        char_probs = self.get_next_char_probabilities()

        for i in range(len(name) - 2):
            char1 = name[i]
            char2 = name[i + 1]
            char3 = name[i + 2]

            if char1 not in char_probs or char2 not in char_probs[char1] or char3 not in char_probs[char1][char2]:

                prob_char1 = 1 / len(self.vocab)
                prob_char2_given_char1 = 1 / len(self.vocab)
                prob_char3_given_char1_char2 = 1 / len(self.vocab)
                log_prob += np.log(lambda_1 * prob_char1 + lambda_2 * prob_char2_given_char1 + lambda_3 * prob_char3_given_char1_char2)
            else:
                log_prob += np.log(char_probs[char1][char2][char3])

        return log_prob



    def get_perplexity(self, text):
        """
        Returns the perplexity of the model on a text as a float.

        Args:
            text [list]: a list of string tokens

        Returns:
            perplexity of the given text [float]
        """

        # ADD YOUR CODE HERE
        log_prob =0.0
        n=0
        for name in text:
            log_prob += self.get_name_log_probability(name)
            n += len(name)

        perplexity = math.exp(-log_prob/n)
        return perplexity


    def sample_next_char (self,name, temperature):
        prev_char = name[-1]
        prev_prev_char = name[-2]

        temp = self.char_probs.get(prev_prev_char, {})

        next_char_probs = temp.get(prev_char,{})

        if temperature > 0.0:
            probs = [p ** (1 / temperature) for p in next_char_probs.values()]
            sum_probs = sum(probs)
            probs = [p / sum_probs for p in probs]
        else:
            probs = next_char_probs.values()

        # Sample a next character
        next_char = random.choices(list(next_char_probs.keys()), weights=probs)[0]

        # Ensure the sampled character is valid
        if next_char not in self.vocab[prev_prev_char][prev_char]:
            next_char = random.choice(list(self.vocab[prev_char]))

        return next_char

    def sample_from_bigram(self,name,temperature):
        unigram_model = BigramModel(train_text)
        return unigram_model.sample_next_char(name,temperature)

    def generate_names(self, k, n=MAX_NAME_LENGTH, prefix=None):
        """
        Given a prefix, generate k names according to the model.
        The default prefix is None.

        Args:
            k [int]: Number of names to generate
            n [int]: Maximum length (number of tokens) in the generated name
            prefix [list of tokens]: Prefix after which the names have to be generated

        Returns:
            list of generated names [list]
        """

        # ADD YOUR CODE HERE
        names= []
        for _ in range(k):
            if prefix:
                if prefix[-1] == '</s>':
                    prefix = prefix[:-1]
                name=prefix
            else:
                name=['<s>']
                next_c = self.sample_from_bigram(name,0.5)
                name.append(next_c)
            while len(name) <n and name[-1]!='</s>':
                next_char = self.sample_next_char(name,0.5)
                if next_char == "<unk>":
                    next_char = self.sample_from_bigram(name,0.5)
                name.append(next_char)
            names.append(name)
        generated_name = [''.join(sublist) for sublist in names]
        return generated_name

    def get_most_likely_chars(self, sequence, k):
        """
        Given a sequence of characters, outputs k most likely characters after the sequence.

        Args:
            sequence [list[str]]: list of characters
            k [int]: number of most likely characters to return

        Returns:
            chars [list[str]]: *Ordered* list of most likely characters
                        (with charcater at index 0 being the most likely and
                        character at index k-1 being the least likely)

        """
        # ADD YOUR CODE HERE
        last_char = sequence[-1]
        last_last_char = sequence[-2]
        next_char_probs = self.char_probs.get(last_last_char, {}).get(last_char,{})
        valid_chars = self.vocab[last_last_char][last_char]
        next_char_probs = {char: prob for char, prob in next_char_probs.items() if char in valid_chars}
        sorted_chars = sorted(next_char_probs, key=next_char_probs.get, reverse=True)
        most_likely_chars = sorted_chars[:k]

        return most_likely_chars



#### Eval

In [ ]:
## Please do not change anything in this code block.

trigram_model = TrigramModel(train_text)

print('trigram train perplexity:',
      trigram_model.get_perplexity(train_text))

trigram train perplexity: 4.298966505162752


In [ ]:
## Please do not change anything in this code block.

eval_ngram_model(model=trigram_model, ngram=3, ds=validation_text, ds_name='validation', eval_prefixes=eval_prefixes, eval_sequences=eval_sequences, num_names=5)

EVALUATION probability distribution is valid: True
EVALUATION of 3-gram on validation perplexity: 5.178486767038496
EVALUATION 3-gram generated names are <s>na</s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>, <s><s><s><s><s><s><s><s><s><s><s>
EVALUATION 3-gram generated names with prefix <s><s>shal</s> are <s><s>shal</s>, <s><s>shala</s>, <s><s>shalakesh, <s><s>shalakesh, <s><s>shalakesh
EVALUATION 3-gram top most likely chars after <s><s>aa are n, s, r, l, m


In [ ]:
# Release models we don't need any more.
del trigram_model

# Module 2: Neural Language Modelling

## 2.1 Neural N-gram Language Model

For this part of the assignment, you should use the GPU (you can do this by changing the runtime of this notebook).

In this section, you will implement a neural version of an n-gram model.  The model will use a simple feedforward neural network that takes the previous `n-1` chars and outputs a distribution over the next char.

You will use PyTorch to implement the model.  We've provided a little bit of code to help with the data loading using [PyTorch's data loaders](https://pytorch.org/docs/stable/data.html)

In [ ]:
# Import the necessary libraries

import math
import time
import random
import os, sys
import json
from functools import partial

from tqdm import tqdm
import torch
import torch.nn as nn

from matplotlib import pyplot as plt
import numpy as np

In [ ]:
## Please do not change anything in this code block.

def collate_ngram(batch, text_pipeline):
    """
    Converts the text in the batch to tokens
    and maps the tokens to indices in the vocab.
    The text in the batch is a list of ngrams
    i.e. if N=3, then text contains 3 tokens in a list
    and batch is a list of such texts.

    Returns:
        batch_input [pytorch tensor]:
            input for n-gram model with size batch_size*(ngram-1)
        batch_output [pytorch tensor]:
            output for n-gram model with size batch_size
    """

    batch_input, batch_output = [], []
    for text in batch:
        token_id_sequence = text_pipeline(text)
        # last token is the output, and
        #  pervious ngram-1 tokens are inputs
        output = token_id_sequence.pop()
        input = token_id_sequence
        batch_input.append(input)
        batch_output.append(output)

    # Convert lists to PyTorch tensors and moves to the gpu (if using)
    batch_input = torch.tensor(batch_input, dtype=torch.long)
    batch_output = torch.tensor(batch_output, dtype=torch.long)
    if USE_CUDA:
        batch_input = batch_input.cuda()
        batch_output = batch_output.cuda()

    return batch_input, batch_output


def get_dataloader(input_text, vocab, ngram, batch_size, shuffle):
    """
    Creates a dataloader for the n-gram model which
    takes in a list of list of tokens, appends the START token
    at the starting of each text, and converts text into ngrams.

    Example: For a trigram model, the list of characters are
        ["n", "a", "v", "r"]
    will be converted into lists
        ["n", "a", "v"], ["a", "v", "r"]

    For each ngram, first ngram-1 tokens are input and last token
    is the output. Each token is converted into a index in the vocab.
    The dataloader generates a batch of input, output pairs as
    pytorch tensors.


    Args:
        input_text [list[list[str]]]: list of list of tokens
        vocab [torchtext.vocab]: vocabulary of the corpus
    """

    ngram_sequences = []
    for text in input_text:
        if text[0] == START:
            text = [START]*(N_GRAM_LENGTH-2) + text
        else:
            text = [START]*(N_GRAM_LENGTH-1) + text

        # Create training pairs for each char in the text
        for idx in range(len(text) - ngram + 1):
            ngram_sequence = text[idx : (idx + ngram)]
            ngram_sequences.append(ngram_sequence)

    text_pipeline = lambda x: vocab(x)
    collate_fn = collate_ngram

    # creates a DataLoader for the dataset

    """
    dataloader documentation
    https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
    """

    dataloader = DataLoader(
        ngram_sequences,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=partial(collate_fn, text_pipeline=text_pipeline),
        )
    return dataloader

#### FNN Implementation

**Feed-forward Neural Language Modelling**

Like the n-gram LM, the feedforward neural LM approximates the probability of a char given the entire prior context $P(w_t|w_{1:t−1})$ by approximating based on the $N-1$ previous chars:
$$P(w_t|w_1,...,w_{t−1}) ≈ P(w_t|w_{t−N+1},...,w_{t−1})$$


Implement the FNN LM given in this paper: [Neural Probabilistic Language Model](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)

The architecture of the FNN can be described by the equation and figure:

$$y = b + W x + U \text t \text a \text n \text h (d + H x)$$

- $x$ is of size $(ngram-1)*m$ where $m$ is the size embedding dimensions
- $y$ is of size $V*1$ where $V$ is the vocabulary size

![FNN_LM](https://drive.google.com/uc?id=1aQhkXjWelHfiBfmBQV3z5TjHFNMtqtzT)


**Some tips**:
- embed the chars with dimension $m$ (example, $60$), then flatten into a single embedding for  $n-1$  chars (with size  $(n-1)*m$ )
- you can use Adam or Stochastic Gradient Descent (SGD) for optimising the cross entropy loss
- If you are using SGD, you may want to use momentum, and a learning rate scheduler
- do early stopping based on validation set loss or perplexity

**Important**: Fix seed as 42 whenever performing any randomized operations, e.g., initializing ML models.

In [ ]:
START = '<s>'
END = '</s>'

In [ ]:
import torch.nn.functional as F

class FNN_LM(nn.Module):

    def __init__(self, vocab_size, emb_size, hid_size, ngram):
        super(FNN_LM, self).__init__()
        self.ngram = ngram

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, emb_size)

        # Bias parameters
        self.output_bias = nn.Parameter(torch.randn(vocab_size, 1))  # Bias for output layer
        self.hidden_bias = nn.Parameter(torch.randn(hid_size, 1))  # Bias for hidden layer

        # Weight matrices
        self.output_weights = nn.Parameter(torch.randn(vocab_size, hid_size))  # Weight matrix for output layer
        self.context_weights = nn.Parameter(torch.randn(vocab_size, (ngram - 1) * emb_size))  # Context weight matrix
        self.hidden_context_weights = nn.Parameter(torch.randn(hid_size, (ngram - 1) * emb_size))  # Hidden layer context weight matrix

        # Additional tensors
        self.e = torch.randn(60, 2)
        self.w = torch.randn(100, 120)

    def forward(self, chars):
        """
        Args:
            chars: this is a tensor of inputs with shape [batch_size x ngram-1]

        Returns:
            logits: a tensor of log probabilities with shape [batch_size x vocab_size]

        """

        embedded = self.embedding(chars).view(chars.size(0), -1)  # Flattening the embeddings
        embedded_t = embedded.t()
        pad_size = self.hidden_context_weights.size(1) - embedded_t.size(0)

        if pad_size > 0:
            embedded_t = F.pad(embedded_t, (0, pad_size))

        hidden_context_output = torch.mm(self.hidden_context_weights, embedded_t)  # Hx
        hidden_context_output = torch.tanh(self.hidden_bias + hidden_context_output)  # tanh(Hx+d)
        hidden_context_output = torch.mm(self.output_weights, hidden_context_output)  # Utanh(Hx+d)

        pad_size = self.context_weights.size(1) - embedded_t.size(0)
        if pad_size > 0:
            embedded_t = F.pad(embedded_t, (0, pad_size))

        context_output = self.output_bias + torch.mm(self.context_weights, embedded_t)  # b+Wx
        combined_output = context_output + hidden_context_output  # b+Wx+Utanh(Hx+d)
        logits = combined_output.t()

        # Additional operation using tensors e and w
        f = torch.flatten(self.e).view(-1, 1)
        y = torch.mm(self.w, f)

        return logits, y


**The following is the Trainer class for the FNN LM. Add your code for the `training` and `validation` loops.**

In [ ]:
!pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 4.7 MB/s eta 0:00:00


In [ ]:
from torcheval.metrics.text import Perplexity
class NeuralNGramTrainer:
    """
    NeuralNGramTrainer wraps FNN_LM to handle training and evaluation.

    """

    # NOTE: you are free to add additional inputs/functions
    # to NeuralNGramTrainer to make training better
    # make sure to define and add it within the input
    # and initialization if you are using any additional inputs
    # for usage in the function

    def __init__(
        self,
        ngram,
        model,
        optimizer,
        criterion,
        train_dataloader,
        valid_dataloader,
        epochs,
        use_cuda,
        vocab,
        model_dir
    ):

        self.ngram = ngram
        self.model = model
        self.epochs = epochs
        self.optimizer = optimizer
        self.criterion = criterion
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.use_cuda = use_cuda
        self.model_dir = model_dir
        self.loss = {"train": [], "val": []}
        self.vocab = vocab

        # Move the model to GPU if available
        if self.use_cuda:
            self.model = self.model.cuda()


    def train(self):
        """
        Trains the model with train_dataloader and validates using valid_dataloader

        """
        # You may change the input arguments to this function,
        # but make sure to also change the code wherever this function is called

        # ADD YOUR CODE HERE
        # FOR TRAINING & VALIDATION
        for epoch in range(self.epochs):
            total_loss = 0
            for batch_input, batch_output in self.train_dataloader:
                if self.use_cuda:
                    batch_input, batch_output = batch_input.cuda(), batch_output.cuda()

                # Zero the gradients
                self.optimizer.zero_grad()

                # Forward pass
                predictions = self.model(batch_input)

                # Calculate loss
                loss = self.criterion(predictions, batch_output)

                # Backward pass
                loss.backward()

                # Update weights
                self.optimizer.step()

                total_loss += loss.item()

            # Calculate average training loss for the epoch
            avg_loss = total_loss / len(self.train_dataloader)
            self.loss['train'].append(avg_loss)

            # Validation
            val_loss = self.evaluate(self.valid_dataloader)
            self.loss['val'].append(val_loss)

            print(f"Epoch {epoch+1}/{self.epochs}, Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}")



    def evaluate(self,dataloader):
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
              for batch_input, batch_output in dataloader:
                  if self.use_cuda:
                      batch_input, batch_output = batch_input.cuda(), batch_output.cuda()

                  predictions = self.model(batch_input)
                  loss = self.criterion(predictions, batch_output)
                  total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        return avg_loss

    def plot_losses(self):
        """
        Plots the training and validation losses
        """
        plt.plot(self.loss['train'], label='train_ppl')
        plt.plot(self.loss['val'], label='val_ppl')
        plt.legend()
        plt.show


    def save_model(self):
        """
        Save final model to directory
        """
        model_path = os.path.join(self.model_dir, "model.pt")
        torch.save(self.model, model_path)

    def save_loss(self):
        """
        Save train/val loss as json file to the directory
        """
        loss_path = os.path.join(self.model_dir, "loss.json")
        with open(loss_path, "w") as fp:
            json.dump(self.loss, fp)


    def get_next_char_probabilities(self):
        """
        Return a dictionary of probabilities for each char in the vocabulary
        with a default starting sequence of [START]*(ngram-1)
        Example:
            If ngram=3, then default starting sequence for which
            probabilities have to be returned is
            [START, START]

        Returns:
            dictionary with key: char, value: probability

        """
        # ADD YOUR CODE HERE
        # Create starting sequence
        self.model.eval()

        prefix = [self.vocab.get_stoi()['<s>']] * (self.ngram - 1)

        prefix = torch.tensor(prefix).unsqueeze(0)


        # print(prefix)
        if self.use_cuda:
            prefix = prefix.cuda()
        logits=self.model(prefix)
        probabilities = torch.softmax(logits, dim=1).squeeze().detach().numpy()
        next_char_probabilities = dict(zip(self.vocab.get_itos(), probabilities))
        return next_char_probabilities




    def generate_names(self, k, n=MAX_NAME_LENGTH, prefix=None):
        """
        Given a prefix, generate k names according to the model.
        The default prefix is None.

        Args:
            k [int]: Number of names to generate
            n [int]: Maximum length (number of tokens) in the generated name
            prefix [list of tokens]: Prefix after which the names have to be generated

        Returns:
            list of generated names [list[str]]
        """

        # ADD YOUR CODE HERE
        self.model.eval()  # Set the model to evaluation mode
        generated_names = []

        with torch.no_grad():
            for _ in range(k):
                if prefix:
                    if prefix[-1] == END:
                        prefix = prefix[:-1]
                    if len(prefix) > self.ngram-1:
                        prefix_cut = prefix[-(self.ngram-1):]
                        prefix_ids = [self.vocab.get_stoi()[char] for char in prefix_cut]
                    else:
                        prefix_ids = [self.vocab.get_stoi()[char] for char in prefix]
                    input_data = torch.tensor(prefix_ids, dtype=torch.long).unsqueeze(0) # Create tensor

                else:
                    start_sequence = [START] * (self.ngram - 1)  # Generate default starting sequence
                    start_sequence_ids = [self.vocab.get_stoi()[char] for char in start_sequence]  # Convert to token IDs
                    input_data = torch.tensor(start_sequence_ids, dtype=torch.long).unsqueeze(0) # Gen extra start tokens
                if self.use_cuda:
                    input_data = input_data.cuda()
                name = []

                for _ in range(n):
                    output = self.model(input_data)
                    probabilities = torch.softmax(output, dim=1).squeeze().cpu().numpy()
                    next_token_id = np.random.choice(len(self.vocab.get_stoi()), p=probabilities)
                    next_char = [char for char, idx in self.vocab.get_stoi().items() if idx == next_token_id][0]
                    if next_char ==END:
                        break
                    name.append(next_char)
                    next_token_id_tensor = torch.tensor([[next_token_id]], dtype=torch.long, device=input_data.device)
                    input_data = torch.cat((input_data[:, 1:], next_token_id_tensor), dim=1)

                if prefix:
                        filtered_prefix = [token for token in prefix if token != '<s>']
                        name = filtered_prefix+name
                generated_names.append(name)
            gen_names = [''.join(sublist) for sublist in generated_names]

            return gen_names



    def get_perplexity(self, text):
        """
        Returns the perplexity of the model on text as a float.

        Args:
            text [list[list[str]]]: list of tokenised names
            > Example:
            [['<s>', 'a', 'a', 'b', 'i', 'd', '</s>'],
            ['<s>', 'a', 'a', 'b', 'i', 'd', 'a', '</s>']]

        Returns:
            perplexity [float]

        """

        # ADD YOUR CODE HERE

        # you may want to use the dataloader here
        # don't forget self.model.eval()
        self.model.eval()
        total_loss = 0.0
        total_tokens = 0
        with torch.no_grad():
          for batch_input, batch_output in train_dataloader:
            output = self.model(batch_input)
            batch_output = batch_output.view(-1)
            loss = self.criterion(output, batch_output)
            total_loss += loss.item() * batch_output.size(0)
            total_tokens += batch_output.size(0)

        avg_loss = total_loss / total_tokens
        perplexity = math.exp(avg_loss)

        # END CODE

        return perplexity



    def get_most_likely_chars(self, sequence, k):
        """
        Given a sequence of characters, outputs k most likely characters after the sequence.

        Args:
            sequence [list[str]]: list of characters
            k [int]: number of most likely characters to return

        Returns:
            chars [list[str]]: *Ordered* list of most likely characters
                        (with charcater at index 0 being the most likely and
                        character at index k-1 being the least likely)

        """

        # ADD YOUR CODE HERE

        # don't forget self.model.eval()

        self.model.eval()
        s = sequence.copy()
        while (len(s)) < self.ngram-1:
          s.insert(0, START)
        most_likely=[]
        current_sequence = [self.vocab.get_stoi()[char] for char in s][-(self.ngram-1):]
        current_sequence_tensor = torch.tensor([current_sequence])
        if self.use_cuda:
            current_sequence_tensor = current_sequence_tensor.cuda()
        #print(current_sequence_tensor)
        logits=self.model(current_sequence_tensor)
        probabilities = torch.softmax(logits, dim=1).squeeze()
        topk_values, topk_indices = torch.topk(probabilities, k)
        for i in topk_indices:
          most_likely.append(self.vocab.get_itos()[i])
        return most_likely


In [ ]:
"""choose your hyperparameter and see the difference in performance"""

# ADD YOUR CODE HERE

# CHANGE THE None VALUES TO YOUR DESIRED VALUES
# Please feel free to play with these hyperparameters to see the effects on the
# quality of generated names and perplexity

MAX_NAME_LENGTH = 10 # maximum length of name for generation

# Remember to fix seed as 42
torch.manual_seed(42)

# check if GPU is available
USE_CUDA = torch.cuda.is_available()
print(f"GPU is available: {USE_CUDA}")

N_GRAM_LENGTH = 5 # The length of the n-gram (N_GRAM_LENGTH=3 for trigram)
EMB_SIZE = 60 # The size of the embedding
HID_SIZE = 120 # The size of the hidden layer
EPOCHS = 10
BATCH_SIZE = 64
SHUFFLE = True # if dataset should be shuffled

GPU is available: False


In [ ]:
## Please do not change anything in this code block.

# Get data iterator and build vocabulary from input text
train_text, vocab = get_tokenised_text_and_vocab(ds_type='train')
validation_text, _ = get_tokenised_text_and_vocab(ds_type='valid', vocab=vocab)

# Check the size of vocabulary
vocab_size = len(vocab.get_stoi())
print(vocab_size)

# Load training and validation dataloaders
train_dataloader = get_dataloader(train_text, vocab, ngram = N_GRAM_LENGTH, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
valid_dataloader = get_dataloader(validation_text, vocab, ngram = N_GRAM_LENGTH, batch_size=BATCH_SIZE, shuffle=SHUFFLE)

131


In [ ]:
# ADD YOUR CODE HERE

# This is the part where you should train your FNN_LM model

# CHANGE THE None VALUES TO YOUR DESIRED VALUES

# Initialise the model, optimizer, learning rate scheduler (optional), and loss criteria
model = FNN_LM(vocab_size=vocab_size, emb_size=EMB_SIZE, hid_size=HID_SIZE, ngram=N_GRAM_LENGTH)
# Move the model to GPU if available
if USE_CUDA:
  model = model.cuda()


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# ADD YOUR CODE HERE
# change the directory name with your SAPname and SRno

model_dir = 'Pritam_Gouda/fnn'

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# NOTE: if you are **optionally** using additional options for the trainer
# (e.g., a training scheduler), please add them below.
trainer = NeuralNGramTrainer(
        ngram=N_GRAM_LENGTH,
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        train_dataloader=train_dataloader,
        valid_dataloader=valid_dataloader,
        epochs=EPOCHS,
        use_cuda=USE_CUDA,
        model_dir=model_dir,
        vocab=vocab)

# Train the model
trainer.train()
print("Training finished.")

trainer.save_model()
trainer.save_loss()
vocab_path = os.path.join(model_dir, "vocab.pt")
torch.save(vocab, vocab_path)
print("Model artifacts saved to folder:", model_dir)

TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not tuple

### Eval

In [ ]:
eval_ngram_model(trainer, ngram=N_GRAM_LENGTH, ds=validation_text, ds_name='valid', eval_prefixes=eval_prefixes, eval_sequences=eval_sequences, num_names=5, is_neural=True)

Load your saved model and generate a few names

In [ ]:
START = "<s>"   # Start-of-name token
END = "</s>"    # End-of-name token
UNK = "<unk>"   # token representing out of unknown (or out of vocabulary) tokens

# ADD YOUR CODE HERE
# change the directory name with your SAPname and SRno

folder = 'Pritam_Gouda/fnn'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the saved model
model = torch.load(f"{folder}/model.pt", map_location=device)
vocab = torch.load(f"{folder}/vocab.pt")

# NOTE: if you are **optionally** using additional options for the trainer
# (e.g., a training scheduler), please add them below.
trainer = NeuralNGramTrainer(
        ngram=N_GRAM_LENGTH,
        model=model,
        optimizer=None,
        criterion=None,
        train_dataloader=None,
        valid_dataloader=None,
        epochs=None,
        use_cuda=USE_CUDA,
        model_dir=None,
        vocab=vocab)

# Generate a few names
names = trainer.generate_names(k=5, n=MAX_NAME_LENGTH, prefix=['a','a','s','h'])
print(", ".join(names))

# you may use this block to test if your model and vocab load properly,
# and that your functions are able to generate sentences, calculate perplexity etc.

In [ ]:
# Release models we don't need any more.
del trainer
del model

## 2.2 Recurrent Neural Networks for Language Modelling

For this stage of the assignment, you will implement an RNN language model.

Some tips:
* use dropout
* use the same weights for the embedding layer and the pre-softmax layer
* train with Adam


In [ ]:

class RNN_LM(nn.Module):

    # you may change the input arguments for __init__
    def __init__(self, vocab_size, emb_size, hid_size, num_layers, dropout):
        super(RNN_LM, self).__init__()


        # YOUR CODE HERE
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.rnn = nn.RNN(emb_size, hid_size, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hid_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        # YOUR CODE HERE
        embedding = self.embedding(x)

        # RNN/LSTM/GRU output and hidden state (if LSTM/GRU)
        output, hidden = self.rnn(embedding)

        # Decode the hidden state to get output logits
        logits = self.fc(output)

        return logits


In [ ]:
class RNNTrainer:
    """
    RNNTrainer wraps RNN_LM to handle training and evaluation.

    """

    # NOTE: you are free to add additional inputs/functions
    # to RNNTrainer to make training better
    # make sure to define and add it within the input
    # and initialization if you are using any additional inputs
    # for usage in the function

    def __init__(
        self,
        model,
        optimizer,
        criterion,
        train_dataloader,
        valid_dataloader,
        epochs,
        use_cuda,
        vocab,
        model_dir
    ):
        self.ngram = 5
        self.model = model
        self.epochs = epochs
        self.optimizer = optimizer
        self.criterion = criterion
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.use_cuda = use_cuda
        self.model_dir = model_dir
        self.loss = {"train": [], "val": []}
        self.vocab = vocab

        # Move the model to GPU if available
        if self.use_cuda:
            self.model = self.model.cuda()

    def train(self):
        """
        Trains the model with train_dataloader and validates using valid_dataloader

        """
        # You may change the input arguments to this function,
        # but make sure to also change the code wherever this function is called

        # ADD YOUR CODE HERE FOR TRAINING & VALIDATION

        for epoch in range(self.epochs):
            self.model.train()  # Set the model to training mode
            train_loss = 0.0

            for batch in self.train_dataloader:
                text, targets = batch
                if self.use_cuda:
                    text, targets = text.cuda(), targets.cuda()

                self.optimizer.zero_grad()
                output = self.model(text)

                # Reshape output and targets for loss calculation
                output = output.view(-1, output.shape[-1])
                targets = targets.view(-1)

                loss = self.criterion(output, targets)
                loss.backward()
                self.optimizer.step()

                train_loss += loss.item() * text.size(0)  # Accumulate with batch size

            epoch_train_loss = train_loss / len(self.train_dataloader.dataset)
            self.loss['train'].append(epoch_train_loss)

            # Validation
            val_loss = self.evaluate()
            self.loss['val'].append(val_loss)

            print(f"Epoch {epoch+1}/{self.epochs}, Train Loss: {epoch_train_loss:.4f}, Val Loss: {val_loss:.4f}")

    def evaluate(self):
        self.model.eval()  # Set the model to evaluation mode
        val_loss = 0.0

        with torch.no_grad():
            for batch in self.valid_dataloader:
                text, targets = batch
                if self.use_cuda:
                    text, targets = text.cuda(), targets.cuda()

                output = self.model(text)
                output = output.view(-1, output.shape[-1])  # Reshape for loss
                targets = targets.view(-1)

                loss = self.criterion(output, targets)
                val_loss += loss.item() * text.size(0)

        return val_loss / len(self.valid_dataloader.dataset)


    def save_model(self):
        """
        Save final model to directory

        """

        model_path = os.path.join(self.model_dir, "model.pt")
        torch.save(self.model, model_path)


    def save_loss(self):
        """
        Save train/val loss as json file to the directory

        """

        loss_path = os.path.join(self.model_dir, "loss.json")
        with open(loss_path, "w") as fp:
            json.dump(self.loss, fp)


    def get_next_char_probabilities(self):
        """
        Return a dictionary of probabilities for each char in the vocabulary
        with a default starting sequence of [START]

        Returns:
            dictionary with key: char, value: probability

        """

        # ADD YOUR CODE HERE
        self.model.eval() #
        start_sequence = [self.vocab.get_stoi()[START]]
        start_tensor = torch.tensor(start_sequence, dtype=torch.long)#

        if self.use_cuda:
            start_tensor = start_tensor.cuda()
        logits = self.model(start_tensor)  # Add a batch dimension
        probs = torch.softmax(logits, dim=1).squeeze().detach().cpu().numpy()  # Probabilities for the last character
        next_char_probabilities = dict(zip(self.vocab.get_itos(), probs))
        return next_char_probabilities


    def generate_names(self, k, n=MAX_NAME_LENGTH, prefix=None):
        """
        Given a prefix, generate k names according to the model.
        The default prefix is None.

        Args:
            k [int]: Number of names to generate
            n [int]: Maximum length (number of tokens) in the generated name
            prefix [list of tokens]: Prefix after which the names have to be generated

        Returns:
            list of generated names [list[str]]
        """

        # ADD YOUR CODE HERE

        # don't forget self.model.eval()


        self.model.eval()  # Set the model to evaluation mode
        generated_names = []

        with torch.no_grad():
            for _ in range(k):
                if prefix:
                    if prefix[-1] == END:
                        prefix = prefix[:-1]
                    if len(prefix)> self.ngram-1:
                        prefix_cut = prefix[-(self.ngram-1):]
                        prefix_ids = [self.vocab.get_stoi()[char] for char in prefix_cut]
                    else:
                        prefix_ids = [self.vocab.get_stoi()[char] for char in prefix]
                    input_data = torch.tensor(prefix_ids, dtype=torch.long).unsqueeze(0) # Create tensor

                else:
                    start_sequence = [START] * (self.ngram - 1)  # Generate default starting sequence
                    start_sequence_ids = [self.vocab.get_stoi()[char] for char in start_sequence]  # Convert to token IDs
                    input_data = torch.tensor(start_sequence_ids, dtype=torch.long).unsqueeze(0) # Gen extra start tokens
                if self.use_cuda:
                    input_data = input_data.cuda()
                name = []

                for _ in range(n):
                # print(tensor) #Got tensor of window size dimensions
                    logits = self.model(input_data)
                    probs = torch.softmax(logits[0, -1], dim=0)
                    next_char_index = torch.multinomial(probs, num_samples=1)
                    next_char = self.vocab.get_itos()[next_char_index]
                    if next_char == END:
                        break
                    name.append(next_char)
                    next_token_id_tensor = torch.tensor([[next_char_index]], dtype=torch.long, device=input_data.device)
                    input_data = torch.cat((input_data[:, 1:], next_token_id_tensor), dim=1)
                if prefix:
                        filtered_prefix = [token for token in prefix if token != '<s>']
                        name = filtered_prefix+name
                generated_names.append(name)
            gen_names = [''.join(sublist) for sublist in generated_names]

            return gen_names




    def get_perplexity(self, text):
        """
        Returns the perplexity of the model on text as a float.

        Args:
            text [list[list[str]]]: list of tokenised names
            > Example:
            [['<s>', 'a', 'a', 'b', 'i', 'd', '</s>'],
            ['<s>', 'a', 'a', 'b', 'i', 'd', 'a', '</s>']]

        Returns:
            perplexity [float]

        """

        # ADD YOUR CODE HERE

        # you may want to use the dataloader here
        # don't forget self.model.eval()

        self.model.eval()  # Set the model to evaluation mode

        total_loss = 0.0
        total_tokens = 0

        with torch.no_grad():
            for batch_input, batch_output in train_dataloader:
                output = self.model(batch_input)
                batch_output = batch_output.view(-1)
                loss = self.criterion(output, batch_output)
                total_loss += loss.item() * batch_output.size(0)  # Multiply by batch size to account for varying token lengths
                total_tokens += batch_output.size(0)

            avg_loss = total_loss / total_tokens
            perplexity = math.exp(avg_loss)


        #perplexity = math.exp(total_loss / total_words)

        return perplexity



    def get_most_likely_chars(self, sequence, k):
        """
        Given a sequence of characters, outputs k most likely characters after the sequence.

        Args:
            sequence [list[str]]: list of characters
            k [int]: number of most likely characters to return

        Returns:
            chars [list[str]]: *Ordered* list of most likely characters
                        (with charcater at index 0 being the most likely and
                        character at index k-1 being the least likely)

        """

        # ADD YOUR CODE HERE

        # don't forget self.model.eval()
        """
        self.model.eval()
        sequence_indices = [self.vocab.get_stoi()[char] for char in sequence]
        sequence_tensor = torch.tensor(sequence_indices).unsqueeze(0)
        if self.use_cuda:
            sequence_tensor = sequence_tensor.cuda()
        output = self.model(sequence_tensor, torch.tensor([len(sequence_indices)]))
        probabilities = torch.softmax(output[:, -1, :], dim=-1).squeeze().cpu().detach().numpy()
        sorted_indices = np.argsort(probabilities)[::-1][:k]
        most_likely_chars = [self.vocab.itos[i] for i in sorted_indices]
        return most_likely_chars
        """
        self.model.eval()


        most_likely=[]
        current_sequence = [self.vocab.get_stoi()[char] for char in sequence]
        # for _ in range(k):
        current_sequence_tensor = torch.tensor(current_sequence)
        if self.use_cuda:
            current_sequence_tensor = current_sequence_tensor.cuda()
        # Forward pass
        logits=self.model(current_sequence_tensor)

        probabilities = torch.softmax(logits, dim=1).squeeze()
        topk_values, topk_indices = torch.topk(probabilities[-1], k)
        most_likely = [self.vocab.get_itos()[idx] for idx in topk_indices.tolist()]
        return most_likely


In [ ]:
from torch.nn.utils.rnn import pad_sequence
def collate_rnn(batch, text_pipeline):

    # Process each sequence in the batch
    batch_input, batch_output = [], []

    for sequence in batch:
        token_id_sequence = text_pipeline(sequence)


        input_sequence = token_id_sequence[:-1]
        output_sequence = token_id_sequence[1:]

        batch_input.append(torch.tensor(input_sequence, dtype=torch.long))
        batch_output.append(torch.tensor(output_sequence, dtype=torch.long))

    batch_input = pad_sequence(batch_input, batch_first=True, padding_value=2)
    batch_output = pad_sequence(batch_output, batch_first=True, padding_value=2)

    # # Convert to PyTorch tensors and move to GPU if necessary
    # batch_input = torch.tensor(batch_input, dtype=torch.long)
    # batch_output = torch.tensor(batch_output, dtype=torch.long)

    if USE_CUDA:
        batch_input = batch_input.cuda()
        batch_output = batch_output.cuda()

    return batch_input, batch_output

def get_rnn_dataloader(input_text, vocab, batch_size, shuffle):
    """
    Creates a dataloader for the RNN-based language model.
    Each sequence in the input_text is treated as a training example.

    Args:
        input_text [list[list[str]]]: list of list of tokens
        vocab [torchtext.vocab]: vocabulary of the corpus
    """

    text_sequences = []
    for sequence in input_text:
        if sequence[0] != START:
            sequence = [START] + sequence  # Add start token at the beginning of each sequence

        text_sequences.append(sequence)

    text_pipeline = lambda x: vocab(x)
    collate_fn = collate_rnn

    dataloader = DataLoader(
        text_sequences,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=partial(collate_fn, text_pipeline=text_pipeline),
    )

    return dataloader

In [ ]:
"""choose your hyperparameter and see the difference in performance"""

# ADD YOUR CODE HERE

# CHANGE THE None VALUES TO YOUR DESIRED VALUES
# Please feel free to play with these hyperparameters to see the effects on the
# quality of generated names and perplexity

MAX_NAME_LENGTH = 10 # maximum length of name for generation

# Remember to fix seed as 42
torch.manual_seed(42)

# check if GPU is available
USE_CUDA = torch.cuda.is_available()
print(f"GPU is available: {USE_CUDA}")

EPOCHS = 10
BATCH_SIZE = 64
SHUFFLE = True # if dataset should be shuffled

# Get data iterator and build vocabulary from input text
train_text, vocab = get_tokenised_text_and_vocab(ds_type='train')
validation_text, _ = get_tokenised_text_and_vocab(ds_type='valid', vocab=vocab)

# Check the size of vocabulary
vocab_size = len(vocab.get_stoi())
print(vocab_size)

# create the dataloaders for training and validation

# ADD YOUR CODE HERE

train_dataloader = get_rnn_dataloader(train_text, vocab, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
valid_dataloader = get_rnn_dataloader(validation_text, vocab, batch_size=BATCH_SIZE, shuffle=SHUFFLE)

In [ ]:
# ADD YOUR CODE HERE

# CHANGE THE None VALUES TO YOUR DESIRED VALUES

# Initialize the model
# you may want to pass arguments to RNN_LM based on your implementation
model = RNN_LM(vocab_size=vocab_size, emb_size=50, hid_size=100,num_layers=2,  dropout=0.5)
# Move the model to GPU if available
if USE_CUDA:
  model = model.cuda()

# Initialise the optimizer, learning rate scheduler (optional), and loss criteria
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# ADD YOUR CODE HERE
# change the directory name with your SAPname and SRno

model_dir = 'Pritam_Gouda/rnn'

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# NOTE: if you are **optionally** using additional options for the trainer
# (e.g., a training scheduler), please add them below.
trainer = RNNTrainer(
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        train_dataloader=train_dataloader,
        valid_dataloader=valid_dataloader,
        epochs=EPOCHS,
        use_cuda=USE_CUDA,
        vocab=vocab,
        model_dir=model_dir
        )

# Train the model
trainer.train()
print("Training finished.")

trainer.save_model()
trainer.save_loss()
vocab_path = os.path.join(model_dir, "vocab.pt")
torch.save(vocab, vocab_path)
print("Model artifacts saved to folder:", model_dir)

### Eval

In [ ]:
## Please do not change anything in this code block.

def eval_rnn_model(model, ds, ds_name, eval_prefixes, eval_sequences, num_names=5):
    """
    Runs the following evaluations on n-gram models:
    (1) checks if probability distribution returned by model.get_next_char_probabilities() sums to one
    (2) checks the perplexity of the model
    (3) generates names using model.generate_names()
    (4) generates names given a prefix using model.generate_names()
    (4) output most likely characters after a given sequence of chars using model.get_most_likely_chars()
    """

    # (1) checks if probability distributions sum to one
    is_valid = check_validity(model, 1, True)
    print(f'EVALUATION probability distribution is valid: {is_valid}')

    # (2) evaluate the perplexity of the model on the dataset
    print(f'EVALUATION of RNN on {ds_name} perplexity:',
        model.get_perplexity(ds))

    # (3) generate a few names
    generated_names = ", ".join(model.generate_names(k=num_names, n=MAX_NAME_LENGTH))
    print(f'EVALUATION RNN generated names are {generated_names}')

    # (4) generate a few names given a prefix
    for prefix in eval_prefixes:
        generated_names_with_prefix = ", ".join(model.generate_names(k=num_names, n=MAX_NAME_LENGTH, prefix=prefix))
        prefix = ''.join(prefix)
        print(f'EVALUATION RNN generated names with prefix {prefix} are {generated_names_with_prefix}')

    # (5) get most likely characters after a sequence
    for sequence in eval_sequences:
        most_likely_chars = ", ".join(model.get_most_likely_chars(sequence=sequence, k=num_names))
        sequence = "".join(sequence)
        print(f"EVALUATION RNN the top most likely chars after {sequence} are {most_likely_chars}")

In [ ]:
t = trainer.generate_names(5,MAX_NAME_LENGTH)
print(t)

In [ ]:
eval_rnn_model(trainer, ds=validation_text, ds_name='valid', eval_prefixes=eval_prefixes, eval_sequences=eval_sequences, num_names=5)

In [ ]:
START = "<s>"   # Start-of-name token
END = "</s>"    # End-of-name token
UNK = "<unk>"   # token representing out of unknown (or out of vocabulary) tokens

# ADD YOUR CODE HERE
# change the directory name with your SAPname and SRno

folder = 'Pritam_Gouda/rnn'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the saved model
model = torch.load(f"{folder}/model.pt", map_location=device)
vocab = torch.load(f"{folder}/vocab.pt")

# NOTE: if you are **optionally** using additional options for the trainer
# (e.g., a training scheduler), please add them below.
trainer = RNNTrainer(
        model=model,
        optimizer=None,
        criterion=None,
        train_dataloader=None,
        valid_dataloader=None,
        epochs=None,
        use_cuda=USE_CUDA,
        model_dir=None,
        vocab=vocab)

# Generate a few names
names = trainer.generate_names(k=5, n=MAX_NAME_LENGTH, prefix=['a','a','s','h'])
print(", ".join(names))

# you may use this block to test if your model and vocab load properly,
# and that your functions are able to generate sentences, calculate perplexity etc.

In [ ]:
# Release models we don't need any more.
del trainer
del model

# Congratulations!

You've reacehd the end of the assignment. Hope this was a pleasant and fun exercise!

Check the submission instructions mentioned at the begining of the notebook in order to submit your homework.
